   It is totally normal to feel like you've been thrown into the deep end here.
   Moving from high-level concepts (like knowing what an ALU is) to actually
   seeing the specific 32-bit physical wiring blueprints (which is what assembly
   formats really are) is a huge jump.

   Think of these "Types" as different puzzle templates. You only have exactly
   32 bits of space to tell the CPU what to do. Depending on the task, you have
   to chop those 32 bits up differently to fit the required pieces.


1. U-Type (Upper Immediate)
   THE CONCEPT: I-Type instructions only give you 12 bits of space to store a
   constant number. But what if you need to load a 32-bit number into a register
   ? You have to do it in two chunks. The U-Type handles the massive top chunk.

   THE BLUEPRINT: It dedicates a whopping 20 bits purely to storing a number
   (`imm[31:12]`), along with a destination register (`rd`) and the `opcode`.

   ASSEMBLY EXAMPLE:
```Assembly
lui x10, 0x87654        # "Load Upper Immediate"
```
   - WHAT IT DOES: It takes the hex number `0x1a65f` (4 bits * 5 placeholders... make sense!)


2. S-Type (Store)

   THE CONCEPT: This is the exact opposite of Load. You have calculated an
   answer in your CPU registers, and now you need to safely store it away in the
   RAM (memory).

   THE BLUEPRINT: Notice in the image that S-Type DOES NOT HAVE A DESTINATION
   REGISTER (`rd`). Why? Because the destination isn't a register; it's the RAM!
   Instead, it uses two source registers (`rs1` and `rs2`) and splits the 
   12-bit immediate into two weird chunks (`imm[11:5]` and `imm[4:0]`). It
   splits the immediate so that `rs1` and `rs2` can stay in the exact same
   physical wire positions as they do in R-Type instructions, which makes the
   hardware engineers' lives easier.

   ASSEMBLY EXAMPLE:
```Assembly
sd x14, 8(x2)           # "Store Doubleword"
```
   - WHAT IT DOES: It takes the data currently sitting in register `x14` (which
     acts as `rs2`) and stores it into the memory address calculated by taking
     the base address in `x2` (`rs1`) and adding an offset of `8`.



3. B-Type (Branch)

   THE CONCEPT: This is your assembly-level `if-statement`. It compares two
   registers, and if a condition is met, it "branches" (jumps) to a different
   line of code.

   THE BLUEPRINT: It looks almost identical to the S-Type. It uses two source
   registers (`rs1` and `rs2`) to compare values. The "immediate" here is the
   offset (how many steps to jump forward or backward). You'll notice the
   immediate bits arte scrambled like crazy (`imm[12]`, `imm[10:5]`, 
   `imm[4:1]`, `imm[11]`). This is a famous RISC-V quirk done purely to make the
   silicon wiring cheaper to manufacture.

   ASSEMBLY EXAMPLE:
```Assembly
beq x19, x10, Label        # "Branch if Equal"
```
   - WHAT IT DOES: The CPU compares `x19` and `x10`. If they hold the exact same
     value, the program counter jumps to wherever `Label` is located in your 
     code.
                  ... huh, how funny.  so the immediate bits are merely so 
                      scrambled in this fashion all because it's cheaper to 
                      manufacture? lolol


4. J-Type (Jump)
   THE CONCEPT: This is an unconditional jump. You aren't checking if two things
   are equal; you are just instantly jumping to a new location. This is how
   function/method calls work in assembly.

   THE BLUEPRINT: Because you might need to jump really far away in your code,
   it dedicates 20 scrambled bits to the jump distance. It also needs a 
   destination register (`rd`). When you jump to a function, you need to 
   remember where to come back to when the function finishes. The `rd` slot
   saves that "return address".


The J-TYPE (Jump) instruction is used for unconditional jumps, meaning the
program skips to a new location in the code without checking a condition first.
In RISC-V, this is primarily used for function calls through the `jal` (Jump and
Link) instruction.


THE BLUEPRINT
   To jump to a new location, the CPU needs two things: the destination address
   and a way to get back. The 32 bits are divided as follows:
   - OPCODE (7 bits): Identifies the instruction as a J-type jump.
   - RD (5 bits): The "Link" register. Before the CPU jumps away, it saves the
     address of the next instruction here so the program knows where to return
     when the function ends.
   - IMMEDIATE (20 bits): This is the "jump distance".

---
THE SCRAMBLED IMMEDIATE
   You might notice in your lecture summary that the bits for the immediate
   (`imm[20]`, `imm[10:1]`, `imm[11]`, `imm[19:12]`) are in a very strange, 
   non-linear order.

   - WHY? This is a hardware optimisation. By placing certain bits in specific
     spots, the physical wires for the `opcode` and `rd` can stay in the same
     place across different instruction types (like U-type), making the chip
     smaller and cheaper to build.


ASSEMBLY EXAMPLE: `jal`
```Assembly
jal x1, 2000        # Jump to the instruction at PC + 2000
```
   
HOW IT WORKS STEP-BY-STEP:
   1. CALCULATE RETURN ADDRESS: The CPU looks at the current PROGRAM COUNTER 
      (PC) and adds 4 (the address of the very next line of code).
   2. LINK: It saves that return address into register `x1`.
   3. JUMP: It takes the scrambled 20-bit immediate, "unscrambles" it, and adds
      it to the current PC.   
   4. EXECUTE: The CPU starts executing the code at that new address.

SUMMARY OF J-TYPE FEATURES:
   - UNCONDITIONAL: It always jumps; it doesn't compare registers like a B-type
     (Branch) instruction.
   - RANGE: Because it has 20 bits for the offset, it can jump much further than
     a B-type instruction (which only has 12 bits). 
   - SAVES PROGRESS: It is designed for subroutines/functions because it "links"
     (saves) the return path.

In [ ]:
  _--_                                     _--_
/#()# #\         0             0         /# #()#\
|()##  \#\_       \           /       _/#/  ##()|
|#()##-=###\_      \         /      _/###=-##()#|
 \#()#-=##  #\_     \       /     _/#  ##=-#()#/
  |#()#--==### \_    \     /    _/ ###==--#()#|
  |#()##--=#    #\_   \!!!/   _/#    #=--##()#|
   \#()##---===####\   O|O   /####===---##()#/
    |#()#____==#####\ / Y \ /#####==____#()#|
     \###______######|\/#\/|######______###/
        ()#O#/      ##\_#_/##      \#O#()
       ()#O#(__-===###/ _ \###===-__)#O#()
      ()#O#(   #  ###_(_|_)_###  #   )#O#()
      ()#O(---#__###/ (_|_) \###__#---)O#()
      ()#O#( / / ##/  (_|_)  \## \ \ )#O#()
      ()##O#\_/  #/   (_|_)   \#  \_/#O##()
       \)##OO#\ -)    (_|_)    (- /#OO##(/
        )//##OOO*|    / | \    |*OOO##\\(
        |/_####_/    ( /X\ )    \_####_\|
       /X/ \__/       \___/       \__/ \X\
      (#/                               \#)

---

In computer architecture, the Register/Registeer (R-Type) instruction is the 
bread and butter of your CPU's mathematical operations. It is used for tasks 
where all the data is already inside the CPU's fast local storage (the registers
) and does not require touching the slower main memory (RAM).


HOW THE R-TYPE BLUEPRINT WORKS
   Because you have exactly 32 BITS to work with, the R-Type format chops the
   instruction into six specific fields to tell the hardware exactly what to do:
      - `opcode` (7 bits): Tells the Control unit that this is a mathematical
        R-Type operation.
      - `rd` (5 bits): The "Destination" register where the final answer will be
        stored.
      - `funct3` && `func7` (10 bits total): These act as sub-codes. While the
        opcode says "do math", these specific bits tell the ALU whether to ADD,
        SUBTRACT, or perform logical shifts.
      - `rs1 && rs2` (5 bits each): These identify the two "Source" registers
        that contain the numbers you want to use for the calculation.


...


SUMMARY OF R-Type Features
   - NO MEMORY ACCESS: Unlike S-Type or I-Type, it never looks at the RAM; it is
     strictly "internal" to the CPU.
   - SPEED: Because it stays within the registers, these are the fastest
     instructions the processor can execute.
   - FIXED SIZE: Like all RISC-V instructions, it is exactly 32 bits long.    


- Yes, RISC-V B-type (Branch) instructions have a relatively small immediate
  range ($\pm$ 4 KiB) primarily due to a combination of fixed 32-bit instruction 
  constraints and the assumption that most control flow changes occur within 
  a 